In [ ]:
import os
import cv2
import config
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from preprocess_data_bb import data_generator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from preprocess_data import read_image_as_array
from preprocess_data import load_train_dataset, load_val_dataset
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, CSVLogger, ReduceLROnPlateau

def train():
    # callbacks
    save_model_callback = ModelCheckpoint(config.model_detector, monitor='val_accuracy', save_best_only=True,
                                          mode='auto', verbose=1, period=1)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', mode='max', restore_best_weights=True, verbose=1)
    #csv_logger_callback = CSVLogger(config.train_log_file, separator=';', append=False)
    #reduce_lr_callback = ReduceLROnPlateau(monitor='val_accuracy', mode='auto', factor=0.5, patience=5, min_lr=0.000001,verbose=1)

    print("[INFO] loading dataset...")
    #print(config.batch_size)
    train_data_gen = data_generator(config.train_dir_bb, config.train_csv, config.batch_size)
    val_data_gen = data_generator(config.val_dir_bb, config.val_csv, config.batch_size)
    
    
    count=0
    for i in val_data_gen:
        print(type(i))
        print(i[1])
        print(count)
        count+=1
    
    # load the VGG16 network
    vgg = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 244, 3)))

    vgg.trainable = False

    flatten = vgg.output
    flatten = Flatten()(flatten)

    bboxHead = Dense(128, activation='relu')(flatten)
    bboxHead = Dense(64, activation='relu')(bboxHead)
    bboxHead = Dense(32, activation='relu')(bboxHead)
    bboxHead = Dense(4, activation='sigmoid')(bboxHead)

    model = Model(inputs=vgg.input, outputs=bboxHead)

    # initialize the optimizer, compile the model and show the model
    opt = Adam(lr=1e-4)
    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    print(model.summary())

    # train the network for bounding box regression
    print("[INFO] training bounding box regress...")
    history = model.fit(x=train_data_gen, validation_data=val_data_gen, batch_size=config.batch_size, epochs=25,
                        verbose=1, callbacks=[save_model_callback, early_stopping_callback])
                                   #reduce_lr_callback, csv_logger_callback])

    #print("[INFO] saving object detector model...")
    #model.save(config.model_detector, save_format='h5')

    #return history


if __name__ == "__main__":
    train()
    #H = train()

    """N = 25
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, N), H.history['loss'], label='train_loss')
    plt.title('Bounding Box Regression Loss on Training Set')
    plt.xlabel('Epoch #')
    plt.ylabel('Loss')
    plt.legend(loc='lower left')
    plt.savefig(config.plot_path_loss)
    
    plt.figure()
    plt.plot(np.arange(0, N), H.history['accuracy'], label='accuracy', color='blue')
    plt.title('Bounding Box Accuracy on Training Set')
    plt.xlabel('Epoch #')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower left')
    plt.savefig(config.plot_path_acc)"""


[INFO] loading dataset...
<class 'tuple'>
[[0.42148829 0.3516367  0.48507032 0.40657032]
 [0.44542384 0.4596133  0.48138475 0.5030449 ]
 [0.63288283 0.33713084 0.681541   0.37383986]
 [0.44553712 0.4893789  0.49074414 0.5268828 ]
 [0.6702266  0.35049218 0.7148203  0.39660546]
 [0.6155449  0.51985353 0.65178514 0.5667246 ]
 [0.11066016 0.8912266  0.15508008 0.92693555]
 [0.42804882 0.38310546 0.49931446 0.43399218]
 [0.5053887  0.4238672  0.5588867  0.47736523]
 [0.49132228 0.28914648 0.6815039  0.41846094]
 [0.3366719  0.27616927 0.37642708 0.31592318]
 [0.50078714 0.35515624 0.65223044 0.45911133]
 [0.5175039  0.29253712 0.66656053 0.40599415]
 [0.5144199  0.35151953 0.68902737 0.45570314]
 [0.488459   0.3305176  0.65794724 0.44410548]
 [0.51725    0.35048437 0.6868711  0.45220703]
 [0.445416   0.38458204 0.49573436 0.42874414]
 [0.5483828  0.3970547  0.67803127 0.48967382]
 [0.286166   0.38858008 0.32045117 0.41997266]
 [0.57249415 0.4096406  0.7729746  0.66657615]
 [0.5461426  0.522

Epoch 1/25
      6/Unknown - 20s 3s/step - loss: 0.0223 - accuracy: 0.4323